In [1]:
import pandas as pd
import numpy as np

In [2]:
luna_label = pd.read_excel('../../data/Luna_training_labels_v2.xlsx')
luna_data = pd.read_csv('../../data/Luna_training_ULT.csv', encoding='utf-16')

In [3]:
# Function to drop columns (tests) in the raw data file which are NOT tests (i.e metadata like start time, lot number etc)
def test_only(datafile,labelfile):
    keys = list(labelfile['Name'])
    df = datafile[keys]
    return df

luna_tests = test_only(luna_data,luna_label)

In [4]:
#Function to find out non applicable rows (no distribution, not part of training data)
def non_applicable (df_label):
    null_rowsLabel = df_label[df_label['Distribution Type'].isnull()]
    null_rowsLabel = list(null_rowsLabel['Name'])
    return null_rowsLabel

luna_unlabelled = non_applicable(luna_label)

In [5]:
luna_target = luna_label[['Name','Distribution Type']]

distribution_counts = luna_target['Distribution Type'].value_counts()
distribution_counts

Distribution Type
functional    2099
outlier        649
longtail       299
discrete       225
normal          90
Untagged        16
bimodal          8
na               6
Name: count, dtype: int64

In [6]:
luna_transposed = luna_tests.T.reset_index(inplace=False, drop=False)
luna_transposed.columns = ['Name'] + [i for i in range(1, len(luna_transposed.columns))]
luna_transposed

,Name,1,2,3,4,5,6,7,8,9,...,10735,10736,10737,10738,10739,10740,10741,10742,10743,10744
0,p_openXbegin_PAD_x_500uA__VBATPWR,-0.485749,-0.485348,-0.485128,-0.485462,-0.485108,-0.486069,-0.485017,-0.484845,-0.485937,...,-0.495930,-0.496403,-0.496009,-0.496189,-0.495989,-0.495285,-0.496782,-0.496654,-0.495755,-0.495535
1,p_openXbegin_PAD_x_500uA__VDDIO,-0.429951,-0.428958,-0.430099,-0.428892,-0.429425,-0.430585,-0.429392,-0.428324,-0.430066,...,-0.450894,-0.450243,-0.448584,-0.450181,-0.449339,-0.447545,-0.450976,-0.450121,-0.449893,-0.448001
2,p_openXbegin_PAD_x_500uA__NFC_GPIO0,-0.658435,-0.656091,-0.657087,-0.657727,-0.660352,-0.660824,-0.657328,-0.657283,-0.658061,...,-0.675915,-0.677388,-0.679115,-0.676605,-0.676481,-0.672854,-0.673171,-0.675772,-0.675584,-0.677225
3,p_openXbegin_PAD_x_500uA__NFC_GPIO1,-0.657163,-0.656260,-0.654594,-0.655976,-0.657814,-0.658664,-0.654103,-0.657050,-0.657026,...,-0.676926,-0.677328,-0.677960,-0.674872,-0.675328,-0.672481,-0.675369,-0.674746,-0.675557,-0.675976
4,p_openXbegin_PAD_x_500uA__NFC_GPIO2_AO,-0.499695,-0.498872,-0.501647,-0.496202,-0.498382,-0.501199,-0.499845,-0.497857,-0.498686,...,-0.520135,-0.517769,-0.518015,-0.519442,-0.517544,-0.514087,-0.516520,-0.518838,-0.516636,-0.513317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,f_nvmXWritePatvformRdtyperdNvm1_x_x_vhsform__r...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,NaN
3388,f_nvmXWritePatvformRdtyperdNvm1_x_x_vhsform__r...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
3389,f_nvmXWritePatvformRdtyperdRedenNvm1_x_x_vhsfo...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
3390,f_nvmXWritePatvformRdtyperdRedenNvm1_x_x_vhsfo...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,NaN


In [7]:
luna_merged = pd.merge(luna_target, luna_transposed, on='Name', how='inner')
df_luna = luna_merged.T

In [8]:
features_luna = df_luna.iloc[2:]
labels_luna = df_luna.iloc[:2]

# Scaling

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
normalized_data = scaler.fit_transform(features_luna)
scaled_features_luna = pd.DataFrame(normalized_data)

/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [10]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
minmax_data = scaler.fit_transform(features_luna)
minmax_features_luna = pd.DataFrame(minmax_data)

/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/_array_api.py:701: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/_array_api.py:718: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [11]:
import importlib
import functions 

importlib.reload(functions)

<module 'functions' from '/Users/snowy/wafers/Luna/functions.py'>

In [12]:
import functions 

features_luna_final = functions.get_features(minmax_features_luna)
features_luna_final

,Count,Unique_Count,Mean,Median,Std_Dev,IQR,Skewness,Kurtosis,Min,Max,...,Outliers_Zscore_prop,Outliers_IQR,Outliers_IQR_prop,Outliers_Tukey,Outliers_Tukey_prop,QQ Count,KS_Stat_norm,KS_P_value_norm,Shapiro_Stat,Shapiro_P_value
0,10744,7003,0.109915,0.112241,0.035912,0.050512,1.605880,41.208402,0.0,1.0,...,0.000279,10744,1.000000,2,0.000186,4,0.030411,4.572208e-09,0.930422,6.981834e-57
1,10744,7498,0.574196,0.593378,0.194096,0.281113,-0.321207,-0.532831,0.0,1.0,...,0.000000,0,0.000000,0,0.000000,210,0.041946,7.297659e-17,0.983791,3.541060e-33
2,10744,7943,0.883541,0.886074,0.044407,0.057402,-1.193837,16.598391,0.0,1.0,...,0.001582,10744,1.000000,7,0.000652,28,0.024858,3.362333e-06,0.958828,1.386030e-47
3,10744,7811,0.535529,0.542743,0.138557,0.186456,-0.200685,-0.340537,0.0,1.0,...,0.000465,10744,1.000000,0,0.000000,232,0.024241,6.453124e-06,0.993951,7.075407e-21
4,10744,7944,0.745949,0.747806,0.042186,0.058650,-0.816421,10.210895,0.0,1.0,...,0.000838,10744,1.000000,5,0.000465,34,0.024024,8.079906e-06,0.968933,6.009823e-43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,10743,11,0.000771,0.000000,0.018579,0.000000,39.944780,1825.788716,0.0,1.0,...,0.004840,52,0.004840,52,0.004840,26,0.511717,0.000000e+00,0.018259,7.440625e-117
3388,10743,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0,0.000000,0,0.000000,0,1.000000,0.000000e+00,0.000000,0.000000e+00
3389,10743,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0,0.000000,0,0.000000,0,1.000000,0.000000e+00,0.000000,0.000000e+00
3390,10743,3,0.000186,0.000000,0.011815,0.000000,70.502825,5366.722498,0.0,1.0,...,0.000279,3,0.000279,3,0.000279,6,0.506007,0.000000e+00,0.003628,2.985696e-117


In [13]:
df_final_luna = pd.concat([labels_luna.T, features_luna_final], axis=1)
df_final_luna

,Name,Distribution Type,Count,Unique_Count,Mean,Median,Std_Dev,IQR,Skewness,Kurtosis,...,Outliers_Zscore_prop,Outliers_IQR,Outliers_IQR_prop,Outliers_Tukey,Outliers_Tukey_prop,QQ Count,KS_Stat_norm,KS_P_value_norm,Shapiro_Stat,Shapiro_P_value
0,p_openXbegin_PAD_x_500uA__VBATPWR,outlier,10744,7003,0.109915,0.112241,0.035912,0.050512,1.605880,41.208402,...,0.000279,10744,1.000000,2,0.000186,4,0.030411,4.572208e-09,0.930422,6.981834e-57
1,p_openXbegin_PAD_x_500uA__VDDIO,longtail,10744,7498,0.574196,0.593378,0.194096,0.281113,-0.321207,-0.532831,...,0.000000,0,0.000000,0,0.000000,210,0.041946,7.297659e-17,0.983791,3.541060e-33
2,p_openXbegin_PAD_x_500uA__NFC_GPIO0,outlier,10744,7943,0.883541,0.886074,0.044407,0.057402,-1.193837,16.598391,...,0.001582,10744,1.000000,7,0.000652,28,0.024858,3.362333e-06,0.958828,1.386030e-47
3,p_openXbegin_PAD_x_500uA__NFC_GPIO1,outlier,10744,7811,0.535529,0.542743,0.138557,0.186456,-0.200685,-0.340537,...,0.000465,10744,1.000000,0,0.000000,232,0.024241,6.453124e-06,0.993951,7.075407e-21
4,p_openXbegin_PAD_x_500uA__NFC_GPIO2_AO,outlier,10744,7944,0.745949,0.747806,0.042186,0.058650,-0.816421,10.210895,...,0.000838,10744,1.000000,5,0.000465,34,0.024024,8.079906e-06,0.968933,6.009823e-43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,f_nvmXWritePatvformRdtyperdNvm1_x_x_vhsform__r...,longtail,10743,11,0.000771,0.000000,0.018579,0.000000,39.944780,1825.788716,...,0.004840,52,0.004840,52,0.004840,26,0.511717,0.000000e+00,0.018259,7.440625e-117
3388,f_nvmXWritePatvformRdtyperdNvm1_x_x_vhsform__r...,functional,10743,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0,0.000000,0,0.000000,0,1.000000,0.000000e+00,0.000000,0.000000e+00
3389,f_nvmXWritePatvformRdtyperdRedenNvm1_x_x_vhsfo...,functional,10743,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0,0.000000,0,0.000000,0,1.000000,0.000000e+00,0.000000,0.000000e+00
3390,f_nvmXWritePatvformRdtyperdRedenNvm1_x_x_vhsfo...,discrete,10743,3,0.000186,0.000000,0.011815,0.000000,70.502825,5366.722498,...,0.000279,3,0.000279,3,0.000279,6,0.506007,0.000000e+00,0.003628,2.985696e-117


In [14]:
# function to relabel target variable into binary values, where outlier and longtail are grouped together
def relabel_data(original_label):
    if original_label in ['outlier', 'longtail']:  
        return 'Outlier/Longtail'
    else:
        return 'Other'

In [15]:
# new column added with the updated labels 
df_final_luna['Target'] = df_final_luna['Distribution Type'].apply(relabel_data)

In [16]:
# encodiing target label into binary variable
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded_test = label_encoder.fit_transform(df_final_luna['Target'])
df_final_luna['Target_Encoded'] = y_encoded_test

In [17]:
# sorting the columns for easier viewing of target labels
reordered_columns = ['Name'] + ['Target'] + ['Target_Encoded'] + [col for col in df_final_luna.columns if col not in ['Target', 'Name', 'Target_Encoded']]
df_final_luna = df_final_luna[reordered_columns]
df_final_luna

,Name,Target,Target_Encoded,Distribution Type,Count,Unique_Count,Mean,Median,Std_Dev,IQR,...,Outliers_Zscore_prop,Outliers_IQR,Outliers_IQR_prop,Outliers_Tukey,Outliers_Tukey_prop,QQ Count,KS_Stat_norm,KS_P_value_norm,Shapiro_Stat,Shapiro_P_value
0,p_openXbegin_PAD_x_500uA__VBATPWR,Outlier/Longtail,1,outlier,10744,7003,0.109915,0.112241,0.035912,0.050512,...,0.000279,10744,1.000000,2,0.000186,4,0.030411,4.572208e-09,0.930422,6.981834e-57
1,p_openXbegin_PAD_x_500uA__VDDIO,Outlier/Longtail,1,longtail,10744,7498,0.574196,0.593378,0.194096,0.281113,...,0.000000,0,0.000000,0,0.000000,210,0.041946,7.297659e-17,0.983791,3.541060e-33
2,p_openXbegin_PAD_x_500uA__NFC_GPIO0,Outlier/Longtail,1,outlier,10744,7943,0.883541,0.886074,0.044407,0.057402,...,0.001582,10744,1.000000,7,0.000652,28,0.024858,3.362333e-06,0.958828,1.386030e-47
3,p_openXbegin_PAD_x_500uA__NFC_GPIO1,Outlier/Longtail,1,outlier,10744,7811,0.535529,0.542743,0.138557,0.186456,...,0.000465,10744,1.000000,0,0.000000,232,0.024241,6.453124e-06,0.993951,7.075407e-21
4,p_openXbegin_PAD_x_500uA__NFC_GPIO2_AO,Outlier/Longtail,1,outlier,10744,7944,0.745949,0.747806,0.042186,0.058650,...,0.000838,10744,1.000000,5,0.000465,34,0.024024,8.079906e-06,0.968933,6.009823e-43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,f_nvmXWritePatvformRdtyperdNvm1_x_x_vhsform__r...,Outlier/Longtail,1,longtail,10743,11,0.000771,0.000000,0.018579,0.000000,...,0.004840,52,0.004840,52,0.004840,26,0.511717,0.000000e+00,0.018259,7.440625e-117
3388,f_nvmXWritePatvformRdtyperdNvm1_x_x_vhsform__r...,Other,0,functional,10743,1,0.000000,0.000000,0.000000,0.000000,...,0.000000,0,0.000000,0,0.000000,0,1.000000,0.000000e+00,0.000000,0.000000e+00
3389,f_nvmXWritePatvformRdtyperdRedenNvm1_x_x_vhsfo...,Other,0,functional,10743,1,0.000000,0.000000,0.000000,0.000000,...,0.000000,0,0.000000,0,0.000000,0,1.000000,0.000000e+00,0.000000,0.000000e+00
3390,f_nvmXWritePatvformRdtyperdRedenNvm1_x_x_vhsfo...,Other,0,discrete,10743,3,0.000186,0.000000,0.011815,0.000000,...,0.000279,3,0.000279,3,0.000279,6,0.506007,0.000000e+00,0.003628,2.985696e-117


In [18]:
# filter rows where any column has null values
null_df = df_final_luna[df_final_luna.isnull().any(axis=1)]

# check the target label (distribution) of these null rows
null_df['Target'].value_counts()

Target
Other    15
Name: count, dtype: int64

In [19]:
df_final_luna_nonull = df_final_luna.dropna() # 15 rows dropped

In [20]:
import pickle

data = {'luna_full': df_final_luna, 'luna_dropped': df_final_luna_nonull}

with open('luna_dataframes.pkl', 'wb') as file:
    pickle.dump(data, file)